# Dane Jordan

## Kaggle Competition - SVC

In [1]:
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import urllib.request

from IPython.core.interactiveshell import InteractiveShell
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [2]:
# load data
x = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/train_features'))
test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
y = np.array(pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/train_labels')))
y = y.astype(float)

# load transformed image data
color_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/color_features'))
compress_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/compress_features'))
crop_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/crop_features'))
crop_to_corner_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/crop_to_corner_features'))
homography_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/homography_features'))
mirror_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/mirror_features'))
rotate30_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/rotate30_features'))
scale_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/scale_features'))

# subset the data (if wanting to use a smaller number of classes)
classes = np.unique(y)
index = np.ravel(np.nonzero(np.in1d(y, classes)))
x_subset = x[index]
y_subset = y[index]

In [15]:
def computeobj(beta, lamb, x, y):
    # number of observations
    n = x.shape[0]
    
    # compute objective function
    obj = (1/n)*np.sum(np.maximum(0, 1 - y*np.dot(x, beta))**2) + lamb*np.sum(beta**2)
    
    return obj


def computegrad(beta, lamb, x, y):
    # number of observations
    n = x.shape[0]
    
    # compute gradient of objective function
    grad_beta = -(2/n)*np.dot(x.T, y*np.maximum(0, 1 - y*np.dot(x, beta))) + 2*lamb*beta
    
    return grad_beta


def backtracking(beta, lamb, x, y, eta=1, alpha=0.5, gamma=0.8, max_iter=100):
    # initialize variables
    grad_beta = computegrad(beta, lamb, x, y)
    norm_grad_beta = np.sqrt(np.sum(grad_beta**2))
    found_eta = 0
    t = 0
    
    # loop through until eta found or max iterations reached
    while found_eta == 0 and t < max_iter:
        if (computeobj(beta - eta*grad_beta, lamb, x, y) <
                computeobj(beta, lamb, x, y) - alpha*eta*norm_grad_beta**2):
            found_eta = 1
        elif t == max_iter:
            break
        else:
            eta = eta*gamma
            t += 1
    
    return eta


def mylinearsvm(beta_init, theta_init, lamb, x, y, max_iter, eps):
    # initialize variables
    beta = beta_init
    theta = theta_init
    grad_theta = computegrad(theta, lamb, x, y)
    eta_init = 1/(max(np.linalg.eigh(np.dot((1/n)*x.T, x))[0]) + lamb)
    beta_vals = [beta_init]
    t = 0
    
    # loop through until EITHER max iterations reached or threshold of epsilon reached
    while t < max_iter and np.linalg.norm(grad_theta) >  eps:
        eta = backtracking(beta, lamb, x, y, eta=eta_init)
        beta_next = theta - eta*grad_theta
        theta = beta_next + t*(beta_next - beta)/(t + 3)
        grad_theta = computegrad(theta, lamb, x, y)
        beta = beta_next
        beta_vals.append(beta)
        t += 1
        
    return beta_vals


def split_data_equal(x, y, test_set, train_size=0.75):
    # split into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0, stratify=y)
    
    # center and standardize x values
    x_scaler = StandardScaler().fit(x_train)
    x_train = x_scaler.transform(x_train)
    x_test = x_scaler.transform(x_test)
    test_set = x_scaler.transform(test_set)
    
    return x_train, x_test, y_train, y_test, test_set


def train_alg(x, y, classes, lamb_list):
    # initialize betas and create list for final beta values
    d = x_train.shape[1]
    beta_init = np.zeros(d)
    theta_init = np.zeros(d)
    final_betas = []
    y_values = []

    # loop through each label and perform ovr appending the final betas for each class
    for i in range(len(classes)):
        lamb = lamb_list[i]
        y_binary = copy.deepcopy(y)
        y_binary[y != classes[i]] = -1
        y_binary[y == classes[i]] = 1
        betas = mylinearsvm(beta_init=beta_init, theta_init=theta_init, lamb=lamb, x=x_train, y=y_binary, max_iter=1000, eps=1e-5)[-1]
        final_betas.append(betas)
        
    return np.array(final_betas)


def predict(x, betas, classes):
    # initialize calculated y values
    y_values = []
    
    # loop through set of final betas and calculate y values
    for i in range(len(betas)):
        y_values.append(np.dot(x, betas[i]))
    
    # calculate predicted values
    y_predict = classes[np.argmax(np.array(y_values), axis=0)]
    
    return y_predict


def accuracy_misclass_error(predict, actual):
    # calculate misclassification error
    misclass_error = np.mean(predict != actual)*100
    accuracy = 100 - misclass_error
    
    return accuracy, misclass_error


def display_confusion_matrix(predict, actual):
    # calculate confusion matrix
    conf_mat = confusion_matrix(y_true=actual, y_pred=predict)
    
    # build visual plot
    plt.matshow(conf_mat);
    plt.title('Confusion Matrix');
    plt.xlabel('Predicted Label');
    plt.ylabel('True Label');
    plt.xticks(range(len(classes)), classes);
    plt.yticks(range(len(classes)), classes);
    
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 18
    fig_size[1] = 12
    plt.rcParams["figure.figsize"] = fig_size
    
    return conf_mat
    

def class_diff(classes, conf_matrix):
    # initialize variable to append each individual class percent
    percent_correct = []
    
    # loop through confusion matrix by true label
    for i in range(len(conf_matrix[0, :])):
        class_count = np.sum(conf_matrix[i])
        misclass_count = 0
        
        # loop through confusion matrix by predict label and append percent correct
        for j in range(len(conf_matrix[:, 0])):
            if i != j:
                misclass_count += conf_matrix[i][j]
            else:
                pass
        percent_correct.append(misclass_count/class_count)
        
    # calcuate ordered list of multi-class misclassification error
    ordered_class_diff = np.vstack((classes, np.array(percent_correct))).T
    ordered_class_diff = ordered_class_diff[ordered_class_diff[:, 1].argsort()[::-1]]
    
    return ordered_class_diff


def export_csv(y, name):
    y = y.astype(int)
    df = pd.DataFrame({'Id':np.arange(1, y.shape[0] + 1), 'Prediction':y})
    df.to_csv(name, sep=',', index=False)
    

def decomp_PCA(train, test, test_set, explained_var_threshold=0.95):
    pca = PCA().fit(train)

    pca_explained_var_ratio = pca.explained_variance_ratio_

    pca_explained_var = []
    num_component_vectors = 0

    while np.sum(pca_explained_var) < explained_var_threshold:
        pca_explained_var.append(pca_explained_var_ratio[num_component_vectors])
        num_component_vectors += 1
    #print('# Component Vectors: %d    Explained Var: %f' % (num_component_vectors, np.sum(pca_explained_var)))

    pca = PCA(n_components=num_component_vectors).fit(train)
    x_train = pca.transform(train)
    x_test = pca.transform(test)
    test_set = pca.transform(test_set)
    
    return x_train, x_test, test_set

## Running sklearn's SVC

In [4]:
# try stacking image tansformations onto the original features
allfeatures = np.vstack((x))
alllabels = np.hstack((y))
test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))

In [5]:
# split data
x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
n = x_train.shape[0]
d = x_train.shape[1]

In [6]:
# run PCA to reduce dimensionality
x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)

In [7]:
'''
Fitting various models is time intensive, especially when trying out multiple sets of features.
For this reason, the code below has been commented out and the allfeatures and alllabels are
set statically at what they were last processed as. Pickle files were used to save the models
and are called in the last cell to show results.
'''

# rbf kernel ovo
# rbfSVC = SVC()
# parameters = {'C':[10**i for i in range(-2, 2)]}
# rbfCV = GridSearchCV(rbfSVC, parameters, n_jobs=-1).fit(x_train, y_train)

# polynomial (order 2) kernel ovo
# poly2SVC = SVC(kernel='poly', degree=2)
# parameters = {'C':[10**i for i in range(-2, 2)]}
# poly2CV = GridSearchCV(poly2SVC, parameters, n_jobs=-1).fit(x_train, y_train)

'\nFitting various models is time intensive, especially when trying out multiple sets of features.\nFor this reason, the code below has been commented out and the allfeatures and alllabels are\nset statically at what they were last processed as. Pickle files were used to save the models\nand are called in the last cell to show results.\n'

In [8]:
# y_predict_rbf = rbfCV.predict(x_test)
# y_predict_poly2 = poly2CV.predict(x_test)
# print('Accuracy: %f%%' % (np.mean(y_predict_rbf == y_test)*100))
# print('Accuracy: %f%%' % (np.mean(y_predict_poly2 == y_test)*100))

## Aggregation of results

In [9]:
# for purposes of efficiency, the models were saved using pickle
all_test_set_predictions_header = ['Id']
all_test_set_predictions = np.arange(1, 4321)

# declare a dictionary for models
my_dict = {}

# image transformations feature sets
feature_sets = ['base',
                'color',
                'compress',
                'crop',
                'crop_to_corner',
                'homography',
                'mirror',
                'rotate30',
                'scale']

#### rbf kernel

In [10]:
# rbf - print all past results for each image transformation added to the base set
print('rbf\nadding image transformations 1 at a time')
for i in feature_sets:
    my_dict['rbf_SVCCV_{0}'.format(i)] = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/SVC/rbf_SVCCV_' + i))
    if i == 'base':
        allfeatures = x
        alllabels = y
    else:
        allfeatures = np.vstack((x, globals()[i + '_set']))
        alllabels = np.hstack((y, y))
    test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
    x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
    x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)
    all_test_set_predictions_header = all_test_set_predictions_header + ['rbf_' + i]
    all_test_set_predictions = np.vstack((all_test_set_predictions, my_dict['rbf_SVCCV_' + i].predict(test_set)))
    print(i, ': %f%%' % (np.mean(my_dict['rbf_SVCCV_' + i].predict(x_test) == y_test)*100))

rbf
adding image transformations 1 at a time
base : 58.055556%
color : 52.037037%
compress : 44.444444%
crop : 72.453704%
crop_to_corner : 51.296296%
homography : 84.259259%
mirror : 87.129630%
rotate30 : 37.314815%
scale : 51.805556%


#### polynomial (order 2) kernel

In [11]:
# polynomial (order 2) - print all past results for each image transformation added to the base set
print('polynomial(order 2)\nadding image transformations 1 at a time')
for i in feature_sets:
    my_dict['poly2_SVCCV_{0}'.format(i)] = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/SVC/poly2_SVCCV_' + i))
    if i == 'base':
        allfeatures = x
        alllabels = y
    else:
        allfeatures = np.vstack((x, globals()[i + '_set']))
        alllabels = np.hstack((y, y))
    test_set = pickle.load(urllib.request.urlopen('https://s3.amazonaws.com/stat558drjordankaggle/test_features'))
    x_train, x_test, y_train, y_test, test_set = split_data_equal(x=allfeatures, y=alllabels, test_set=test_set, train_size=.75)
    x_train, x_test, test_set = decomp_PCA(train=x_train, test=x_test, test_set=test_set)
    all_test_set_predictions_header = all_test_set_predictions_header + ['poly2_' + i]
    all_test_set_predictions = np.vstack((all_test_set_predictions, my_dict['rbf_SVCCV_' + i].predict(test_set)))
    print(i, ': %f%%' % (np.mean(my_dict['poly2_SVCCV_' + i].predict(x_test) == y_test)*100))

polynomial(order 2)
adding image transformations 1 at a time
base : 67.037037%
color : 60.925926%
compress : 51.805556%
crop : 83.750000%
crop_to_corner : 59.629630%
homography : 90.925926%
mirror : 92.685185%
rotate30 : 43.379630%
scale : 62.361111%


#### output all predictions

In [12]:
df = pd.DataFrame(all_test_set_predictions.astype(int).T)
df.to_csv('All_SVC_Predictions.csv', sep=',', header=all_test_set_predictions_header, index=False)